In [43]:
import sqlalchemy as sa
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from IPython.display import display
import requests
import re

# This notebook is thought to be executed one at a time.
# Please read and execute from top to bottom in order.

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
#                                                                   #
#         Configure here your parameters for the analysis           #
#                                                                   #
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

# Goerli
genesis_unix_time = 1616508000 		# Constant
start_epoch = 175200 				# Start epoch you want to plot
start_time = "2023-05-11T06:00:00Z" # Start time from where you want to download into CSV

end_epoch = 176200 					# End epoch you want to plot
end_time = "2023-05-15T18:40:00Z"	# End time from where you want to download into CSV
duration = "107h"					# Duration from start to end (download)
network = "goerli"					# Tag

# Mainnet
genesis_unix_time = 1606824023 		# Constant
start_epoch = 201700  				# Start epoch you want to plot
start_time = "2023-05-16T22:40:00Z"	# Start time from where you want to download into CSV

end_epoch = 202700					# End epoch you want to plot
end_time = "2023-05-21T09:20:23Z"	# End time from where you want to download into CSV
duration = "107h"					# Duration from start to end (download)
network="mainnet"					# Tag

# We had some issues with the Teku instance so we had to relaunch it
teku_start_time = "2023-05-26T09:20:00Z"
teku_end_time = "2023-05-28T07:20:00Z"

# Common
prometheus_ip = "localhost"				# From where to download Prometheus data
project_name = "resource-analysis-v2"	# Tag
step = "1s"								# Step to download data (you might need to increase if too much data: Prometheus allows 11K points)
csv_folder = f"""./{network}/csv/"""	# Where to store CSV files (download data)
fig_folder = f"""./{network}/fig/"""	# Where to store figures (plots)
epoch_seconds = 384						# Constant


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
#                                                                   #
#                  End of cofiguration parameters				    #
#                                                                   #
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

def ask(query_str):
	response = requests.get('http://' + prometheus_ip + ':9091/api/v1/query', params={
		'query': query_str,
		'time': end_time}) 
	response_json = response.json()
	response_data = response_json['data']['result']
	return response_data

def ask_range(query_str, i_start_time, i_end_time):
	response = requests.get('http://' + prometheus_ip + ':9091/api/v1/query_range', params={
		'query': query_str,
		'start': i_start_time,
		'end': i_end_time,
		'step': step}) 
	response_json = response.json()
	response_data = response_json['data']['result']
	return response_data




In [40]:

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
#                                                                   #
#                          Query Definition				            #
#                                                                   #
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

# In our case we had to assign a client tag for each client depending on which IP the node was hosted


ip_clients_map = {
	'0.0.0.0': 'Prysm',
	'0.0.0.0': 'Lighthouse',
	'0.0.0.0': 'Teku',
	'0.0.0.0': 'Nimbus',
	'0.0.0.0': 'Lodestar',
}
general_filter = {
    'project_name': 'resource-analysis-v2',
    'network': network,
    'server': "0.0.0.0|0.0.0.0|0.0.0.0|0.0.0.0|0.0.0.0"
}

metrics = [
	{
	'group': 'cpu',
	'name': 'cpu_seconds_user',
	'ylabel': 'Percentage',
    'ylim': [0, 100],
    'xlabel': 'Epoch',
    'title': 'CPU consumption',
    'queries': [
			{
				'query': f"""rate(node_cpu_seconds_total[{step}]) * 100""",
				'filters': {
					'mode': 'user'
					}
			}
		],
	},
	{
	'group': 'memory',
	'name': 'used_mem',
    'ylabel': 'Percentage',
    'ylim': [0, 100],
    'xlabel': 'Epoch',
    'title': 'Used Memory',
    'queries': [
			{
				'query': f"""(node_memory_MemTotal_bytes - node_memory_MemAvailable_bytes) / node_memory_MemTotal_bytes * 100""",
				'filters': {}
			}
		],
	},
    {
	'group': 'network',
	'name': 'network_receive',
    'ylabel': f"""MB/{step}""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Network Receive Rate',
    'queries': [
			{
				'query': f"""rate(node_network_receive_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
					'device': 'eno1'
					}
			}
		],
	},
    {
	'group': 'network',
	'name': 'network_transmit',
    'ylabel': f"""MB/{step}""",
    'ylim': [0, 2],
    'xlabel': 'Epoch',
    'title': 'Network Send Rate',
    'queries': [
			{
				'query': f"""rate(node_network_transmit_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
					'device': 'eno1'
					}
			}
		],
	},
    {
	'group': 'peers',
	'name': 'libp2p_peers',
    'ylabel': f"""Count""",
    'ylim': [0, 200],
    'xlabel': 'Epoch',
    'title': 'Lib P2P Peers',
    'queries': [
			{
				'query': f"""libp2p_peers""",
				'filters': {}
			},
            {
				'query': f"""p2p_peer_count""",
				'filters': {
                    'state': 'Connected'
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_write_bytes',
    'ylabel': f"""MB/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Write Bytes Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_written_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_read',
    'ylabel': f"""MB/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Read Bytes Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_read_bytes_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_read_ops',
    'ylabel': f"""OPS/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Read Ops Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_reads_completed_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
    {
	'group': 'disk',
	'name': 'disk_write_ops',
    'ylabel': f"""OPS/m""",
    'ylim': [0, 10],
    'xlabel': 'Epoch',
    'title': 'Disk Write Ops Ratio',
    'queries': [
			{
				'query': f"""rate(node_disk_writes_completed_total[{step}]) / 1024 / 1024""",
				'filters': {
                    'device': "nvme0n1"
				}
			}
		],
	},
]


# Metrics

# METRIC_MEMORY = 'node_memory_MemAvailable_bytes'
# METRIC_MEMORYTOTAL = 'node_memory_MemTotal_bytes'
# METRIC_CPU = 'node_cpu_seconds_total'
# METRIC_NETRECV = 'node_network_receive_bytes_total'
# METRIC_NETSENT = 'node_network_transmit_bytes_total'
# METRIC_NETRECV_PACK = 'node_network_receive_packets_total'
# METRIC_NETSENT_PACK = 'node_network_transmit_packets_total'
# METRIC_SLOT = 'beacon_head_slot'
# METRIC_PEERS = 'libp2p_peers'
# METRIC_DISKTOTAL = "node_filesystem_size_bytes"
# METRIC_DISKAVAIL = "node_filesystem_avail_bytes"
# METRIC_DISKWRITE_BYTES = "node_disk_written_bytes_total"
# METRIC_DISKREAD_BYTES = "node_disk_read_bytes_total"
# METRIC_DISKWRITE_OPS = "node_disk_writes_completed_total"
# METRIC_DISKREAD_OPS = "node_disk_reads_completed_total"
# METRIC_EARNS = "validators_epoch_earned_amount_metrics"



In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
#                                                                   #
#                          Data Collection				            #
#                                                                   #
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

df_result = []

for metric in metrics:
	print("Requesting metric: ", metric)
	tmp_csv = []
	for query in metric['queries']:
		
		response_data = ask_range(query['query'], start_time, end_time)
		

		for item in response_data:
			pass_filter = True
			# dismiss wrong metrics
			filters = general_filter | query['filters']
			for key, value in filters.items():
				if key in item['metric']:
					requested = item['metric'][key]
					if not re.match(value, requested):
						pass_filter = False # if any filter does not match, do not include this series
				else:
					pass_filter = False
				
				
			if pass_filter:
				for value in item['values']:
					ip = item['metric']['server']
					tuple = [str(ip_clients_map[ip]), str(value[0]), float(value[1])]
					tmp_csv.append(tuple)

		# Teku exceptional case
		if network == 'mainnet':
			response_data_teku = ask_range(query['query'], teku_start_time, teku_end_time)
			for item in response_data_teku:
				pass_filter = True
				# dismiss wrong metrics
				filters = general_filter | query['filters']
				if item['metric']['server'] != "0.0.0.0":
					continue
				for key, value in filters.items():
					if key in item['metric']:
						requested = item['metric'][key]
					else:
						pass_filter = False
					
					if not re.match(value, requested):
						pass_filter = False # if any filter does not match, do not include this serie
				if pass_filter:
					for value in item['values']:
						ip = item['metric']['server']
						tuple = [str(ip_clients_map[ip]), str(value[0]), float(value[1])]
						tmp_csv.append(tuple)
	
	df = pd.DataFrame(tmp_csv, columns=['client', 'timestamp', 'metric'])
	df = df.groupby(by=['client', 'timestamp']).agg({
		'metric': 'mean'
	}).rename(columns={"metric": metric['name']})
	

	if len(df_result) == 0:
		df_result = df
	else:
		df_result = df_result.merge(df, how='outer', on=['client', 'timestamp'])
df_result = df_result.reset_index()
df_result['timestamp'] = df_result['timestamp'].astype(float)
df_result['epoch'] = ((df_result['timestamp'] - float(genesis_unix_time)) / float(epoch_seconds)).astype(int)
for client in ip_clients_map.values():
	df_client=df_result[df_result.client == client]
	df_client.to_csv(f"""{csv_folder}/{client}.csv""")
		

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
#                                                                   #
#                          Data Plotting				            #
#                                                                   #
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #

colors = {
    'Prysm': 'red',
    'Lighthouse': 'blue',
    'Teku': 'orange',
    'Nimbus': 'green',
    'Lodestar': 'purple'
}

df_prysm = pd.read_csv(f"{csv_folder}/Prysm.csv")
df_lighthouse = pd.read_csv(f"{csv_folder}/Lighthouse.csv")
df_teku = pd.read_csv(f"{csv_folder}/Teku.csv")
df_nimbus = pd.read_csv(f"{csv_folder}/Nimbus.csv")
df_lodestar = pd.read_csv(f"{csv_folder}/Lodestar.csv")

if network == 'mainnet': 
	df_teku['timestamp'] = df_teku['timestamp'].apply(lambda x: (x - 432000) if x > 1684660800 else x)
	df_teku.drop(df_teku[(df_teku['timestamp'] > 1684438620) & (df_teku['timestamp'] < 1684604340)].index, inplace = True)
	df_teku['timestamp'] = df_teku['timestamp'].apply(lambda x: (x - 165720) if x > 1684438620 else x)

	df_teku['epoch'] = ((df_teku['timestamp'] - float(genesis_unix_time)) / float(epoch_seconds)).astype(int)

dfs = [df_prysm, df_lighthouse, df_teku, df_nimbus, df_lodestar]
df_all = pd.concat(dfs)
dfs.append(df_all)
markers_dict = {"Prysm": ".", "Lighthouse": "1", "Teku": "2", "Nimbus": "3", "Lodestar": "4"}

for metric in metrics:
    print("Analyzing metric: ", metric)
    for df, client_name in zip(dfs, ['prysm', 'lighthouse', 'teku', 'nimbus', 'lodestar', 'all']):
        fig = plt.figure(figsize=(15, 8))
        g = sns.scatterplot(data=df, x="epoch", y=metric['name'], 
                            hue='client', 
                            palette=colors,
                            style='client')
        g.set(xlabel=metric['xlabel'], ylabel=metric['ylabel'], title=f"""{metric['title']} ({network})""", ylim=metric['ylim'])
        fig.tight_layout()
        fig.savefig(f"""{fig_folder}/{metric['name']}_{client_name}""")
        plt.close(fig)
        